# Manufacturing Process Optimization Project using SQL

In this project, we'll examine a hypothetical dataset using SQL window functions to identify problems in a manufacturing process. 

The approach this project focuses on is the application of Statistical Process Control (SPC) a proven methodology that leverages data to assure process efficiency.

SPC enables us to define "control limits" that guide our process adjustments. For this project, we'll be using two control limits:

**Upper Control Limit** (UCL): Ensures values (in this case we're using height for demonstration purposes) do not exceed what is considered normal variation, which is defined as:

- $ucl = avg\_height + 3 * \frac{stddev\_height}{\sqrt{5}}$

And, **Lower Control Limit** (LCL): Which likewise, ensures values do not fall below the standard of normal variation.

- $lcl = avg\_height - 3 * \frac{stddev\_height}{\sqrt{5}}$


Our SQL code will operate with these limits and allow us to visualize outliers in our dataset. This will ensure the manufacturing process remains efficient and produces products of consistent quality.

## The data
The data is available in the `manufacturing_parts` table which has the following fields:
- `item_no`: the item number
- `length`: the length of the item made
- `width`: the width of the item made
- `height`: the height of the item made
- `operator`: the operating machine

## Setting up the Environment

We start by preparing our workspace, loading necessary libraries, and connecting to the database. It's the foundation for our SQL journey, so it's important to verify our database connection details are all set.


In [1]:
import os
from sqlalchemy import create_engine

# Set your database URL without credentials as an environment variable or directly in code
db_url = os.getenv('DATABASE_URL', 'postgresql://localhost:5432/postgres')

# Access the username and password from environment variables
db_username = os.getenv('DB_USERNAME')
db_password = os.getenv('DB_PASSWORD')

# Combine all parts to form the full database URL
full_db_url = f'{db_url.replace("postgresql://", f"postgresql://{db_username}:{db_password}@")}'

# Create the database engine
engine = create_engine(full_db_url)


## Loading the Data

Next up, we bring our data into the picture by reading it into a pandas DataFrame from a CSV file. This is our first glance at the dataset, preparing us for the deeper dive ahead with SQL.

In [2]:
import pandas as pd

# Load our data into a DataFrame
data_frame = pd.read_csv('manufacturing_parts.csv')


## Creating the Database Table

With our data in a DataFrame, we now transition it to a SQL table. This step bridges our work from pandas to SQL, setting the stage for some serious data analysis.

In [3]:
# Assuming 'manufacturing_parts' is the table name you want to use
data_frame.to_sql('manufacturing_parts', con=engine, if_exists='replace', index=False)


500

## Executing the SQL Query

Here's where the magic happens. Our SQL query calculates control limits to spot any outliers in our manufacturing data. It's a key step in using data to pinpoint where our process might be veering off track.


In [4]:
from sqlalchemy import text

# Your SQL query
query = text("""
SELECT
    b.*,
    CASE
        WHEN b.height NOT BETWEEN b.lcl AND b.ucl THEN TRUE ELSE FALSE END as alert
FROM (
    SELECT
        a.*,
        a.avg_height + 3*a.stddev_height/SQRT(5) AS ucl,
        a.avg_height - 3*a.stddev_height/SQRT(5) AS lcl
    FROM (
        SELECT
            operator,
            ROW_NUMBER() OVER w,
            height,
            AVG(height) OVER w AS avg_height,
            STDDEV(height) OVER w AS stddev_height
        FROM manufacturing_parts
        WINDOW w AS (
            PARTITION BY operator ORDER BY item_no
            ROWS BETWEEN 4 PRECEDING AND CURRENT ROW
        )
    ) AS a
) AS b
""")

# Execute the SQL query
with engine.connect() as connection:
    result = connection.execute(query).fetchall()


## Analyzing the Results

After running our query, we dissect the results to uncover insights. Identifying outliers is more than just numbers; it's about ensuring quality and consistency in what we produce.


In [5]:
column_names = ['Operator', 'Item_No', 'Height', 'Avg_Height', 'Stddev_Height', 'UCL', 'LCL', 'Alert']

# Create the dataframe
df = pd.DataFrame(result, columns=column_names)

filtered_df = df[df['Alert'] == True]
print(filtered_df)

    Operator  Item_No  Height  Avg_Height  Stddev_Height        UCL  \
16      Op-1       17   19.33      20.092       0.456366  20.704279   
20      Op-1       21   19.94      19.312       0.417457  19.872077   
22      Op-1       23   22.31      20.042       1.342747  21.843484   
38     Op-10       13   18.58      20.482       1.231105  22.133701   
43     Op-10       18   21.86      20.022       1.161215  21.579933   
55     Op-10       30   18.64      20.374       1.248091  22.048489   
63     Op-11        8   18.54      20.346       1.163284  21.906709   
79     Op-12        8   20.67      19.380       0.849706  20.520000   
96     Op-13        5   21.93      20.182       1.186221  21.773482   
107    Op-13       16   21.60      20.250       0.970490  21.552048   
109    Op-13       18   18.88      20.390       1.043144  21.789525   
118    Op-14        6   18.80      20.490       1.202788  22.103710   
126    Op-14       14   21.60      19.676       1.163714  21.237286   
133   